In [ ]:
input_data_path = ''  #Input non-normalizad data
batch = ''
label = ''
epochs = 0

Jupyter notebook for scanvi integration algorithm

Author: Erno Hänninen

Title: run_scanvi.ipynb

Created: 2022-11-29

In [ ]:
import sys
sys.path.insert(0, "../../../../Scripts") #Adding a path to be able to import the jupyter_functions
from jupyter_functions import *
import scanpy as sc
import scib

In [ ]:
#Read the adata object and extract the variable genes to list
adata = sc.read(input_data_path)
hvgList = adata.var.index.tolist()
print(len(hvgList))

In [ ]:
#Run integration algorithm
adata_scanvi, model = scib.integration.scanvi(adata, batch=batch, labels=label, hvg=hvgList, max_epochs=epochs)
print(adata_scanvi)

In [ ]:
train_elbo = model.history['elbo_train'][1:]
train_elbo.plot()

In [ ]:
#using pymde package to visualize the scVI's learned embeddings, like it was done in scvi-tools tutorial

from scvi.model.utils import mde
import pymde
adata_scanvi.obsm["X_mde"] = mde(adata_scanvi.obsm["X_emb"])

sc.pl.embedding(
    adata_scanvi,
    basis="X_mde",
    color=[batch, label],
    frameon=False,
    ncols=1,
)

In [ ]:
#Calling plot function from jupyter_functions.py file
#The function initializes the adata  to be plotted
adata_scanvi_final, basis = plot_results(adata_scanvi,"scanvi","full")
print(adata_scanvi_final)
adata_scanvi_final.write("../Integrated_adata/scanvi_adata_final.h5ad")

In [ ]:
# Plot the results
if batch in adata_scanvi_final.var_names or batch in adata_scanvi_final.obs.columns:
    sc.pl.embedding(adata_scanvi_final, basis=basis, color=batch)
    
if label in adata_scanvi_final.var_names or label in adata_scanvi_final.obs.columns:
    sc.pl.embedding(adata_scanvi_final, basis=basis, color=label)

In [ ]:
#Call the compute metrics function which computes the integration metrics
df = compute_metrics("scanvi", adata, adata_scanvi_final, batch, label,"X_pca", "full")
print(df)  